In [350]:
import requests
import hashlib
from datetime import datetime
import time
import random
import os
import csv
from bs4 import BeautifulSoup
import re 
import pandas as pd

# Parsing functions

In [351]:
def parse_listing_html(text,file_name):
    #define the html text
    page_html = BeautifulSoup(text, 'html.parser')
    
    if deleted_listing(page_html):
        print('listing {} has been deleted'.format(file_name))
        return
    
    if page_html.find('meta',attrs={'property':'og:url'}) is None: 
        return
    else:
        listing_url = page_html.find('meta',attrs={'property':'og:url'}).attrs['content']
    try:
        listing_id = page_html.findAll('p', class_='postinginfo')[1].text[9:]

        listing_title = page_html.find('span',attrs={'id':'titletextonly'}).text
        
        listing_nh = page_html.find('small').text[2:-1]
        
        listing_city = page_html.find('meta',attrs={'name':'geo.placename'}).attrs['content']
        
        listing_date = page_html.find('time', class_='date timeago')['datetime']
        
        if page_html.find('span', class_='price') is None: 
            return
        else:
            listing_price = int(page_html.find('span', class_='price').text.strip('$,').replace(",",""))

        
        listing_bedrooms = page_html.find('span', class_='shared-line-bubble').text.split('BR')[0]
        
        listing_bathrooms = page_html.find('span', class_='shared-line-bubble').text.split('/')[1][:-2]
        
        # Extract sqft info from bubbles class and use regex to find the digits ending in ft2
        bubbles_sqft = page_html.findAll('span', class_='shared-line-bubble')
        listing_sqft = None
        if len(bubbles_sqft) >= 2:
            x = re.search("[0-9]+ft2$", bubbles_sqft[1].text)
            if x:
                 listing_sqft = x.string[:-3]

        listing_address_element = page_html.find('div', class_='mapaddress')
        listing_address = listing_address_element and listing_address_element.text

        listing_info = page_html.findAll('p', class_='attrgroup')[1].text

        listing_body = page_html.find('section',attrs={'id':'postingbody'}).text[30:]

        listing_first_image = page_html.find('div',class_='slide first visible').img['src']
    except Exception as e:
        print('Error parsing url: {} file: {}'.format(listing_url, file_name))
        raise e

    return {
        'listing_title': listing_title,
        'listing_nh': listing_nh,
        'listing_city':listing_city,
        'listing_date':listing_date,
        'listing_price': listing_price,
        'listing_bedrooms':listing_bedrooms,
        'listing_bathrooms':listing_bathrooms,
        'listing_sqft':listing_sqft,
        'listing_address':listing_address,
        'listing_info':listing_info,
        'listing_body':listing_body,
        'listing_id':listing_id,
        'listing_url':listing_url,
        'listing_first_image':listing_first_image
       }


def deleted_listing(page_html):
    h2 = page_html.find('h2')
    return h2 and h2.text == ' This posting has been deleted by its author. '

# Create df from dictionary

In [352]:
list_of_dicts = []
path = "/Users/pandabear/springboard/CapstoneTwoProject/data/raw"
dir_list = os.listdir(path)
#print(dir_list)
for folder in dir_list:
    if not folder.startswith('.'):
        path_html = path + '/' + folder
        file_list = os.listdir(path_html)
        #print(file_list)
        
        for file in file_list:
            if len(file) > 15:
                with open(path_html + '/' + file, 'r') as f:
                    text = f.read()
                    parsed_data = parse_listing_html(text, file)
                    if parsed_data:
                        list_of_dicts.append(parsed_data)
#                 print('Number of parsing errors: {}'.format(count_parsing_error))
        listing_df = pd.DataFrame(list_of_dicts)


listing c3ef226a1bf4ef4080a4b69aa5f663808dd609cefef0feb5804501fa9be01a9c has been deleted
listing 3f4ce9c85d102d6da2ab87fd34c38d797a80832f93d1c13acd0bcf9f29d4e03b has been deleted
listing cd54be649ad10badbfcb63bd4c4a368b548dbd7da493464a726549658f9b5968 has been deleted
listing 00c0b60fca283730cb2b417f04a85b2ac0bf4c935d37f62166100dfec3e4afa5 has been deleted
listing 58b018f9e3542e661ebb75709ce2db0adcd675115af6cdba3bbedd2b50843bf2 has been deleted
listing 075eabcec6f68958678c15ca0effab9e78a03036ff3ddcba03df92056b6c4295 has been deleted
listing a9664a79bb9b68fdf7fc2955bc0ef0678ee35b2840e3c80c1e485071e72d07db has been deleted
listing 405f2f89c11fb5d37f6100405a50307385ec2ecd2e945555c9b43b614a92c5f7 has been deleted
listing 202559b57c80cbb3022b36004db10a4db58241e4df8461723e5099564a8e08f2 has been deleted
listing a489473984c40d69ad8153bc5be0e2f341d309802f2d49c417a66fa684529705 has been deleted
listing b3543edce1dc860b3ba00d1dc7ff41999bc88113b0732412594e18fda9524694 has been deleted
listing bb

In [356]:
listing_df.set_index('listing_id', inplace = True)
listing_df.head()

KeyError: 'listing_id'

In [357]:
listing_df.shape

(57063, 13)

In [358]:
listing_df.to_csv(r'/Users/pandabear/springboard/CapstoneTwoProject/data/interim/listing_df.csv', header=True)
print('Saved to csv file')

Saved to csv file
